In [1]:
# Import necessary libraries
import pandas as pd
from datetime import datetime, timedelta

In [2]:
# Define URLs for datasets
transactions_url = 'https://websdk-assets.s3.ap-south-1.amazonaws.com/public/txns+(13).csv'
repayments_url = 'https://websdk-assets.s3.ap-south-1.amazonaws.com/public/repayments+(2).csv'

In [3]:
# Load datasets using pandas
transactions = pd.read_csv(transactions_url)
repayments = pd.read_csv(repayments_url)


In [4]:
transactions.head(10)


,transactionId,day,downPaymentAmount,emiPrincipal,emiInterest,tenure
0,704663,2023-04-16T00:00:00Z,59950,59950,0,2
1,1403240,2024-02-22T00:00:00Z,33300,33300,0,2
2,1961345,2024-04-21T00:00:00Z,238634,238633,0,2
3,1682956,2024-03-31T00:00:00Z,279934,279933,0,2
4,2392323,2024-06-05T00:00:00Z,140566,140567,0,2
5,1187757,2024-01-10T00:00:00Z,484920,484920,0,2
6,1013712,2023-11-19T00:00:00Z,38473,38471,0,2
7,1047069,2023-12-02T00:00:00Z,363268,363266,0,2
8,2183694,2024-05-16T00:00:00Z,45766,45767,0,2
9,1432937,2024-02-27T00:00:00Z,110910,110910,0,2


In [6]:
repayments.head(10)

,transactionId,repaymentDay,amount
0,19009,2022-02-05T00:00:00Z,93300
1,19009,2022-03-08T00:00:00Z,93300
2,38246,2022-03-26T00:00:00Z,266667
3,40966,2022-04-03T00:00:00Z,76667
4,19009,2022-04-07T00:00:00Z,93300
5,45503,2022-04-15T00:00:00Z,233334
6,46954,2022-04-19T00:00:00Z,150000
7,38246,2022-04-27T00:00:00Z,266667
8,52501,2022-04-28T00:00:00Z,28300
9,56928,2022-05-07T00:00:00Z,333334


In [7]:
# Ensure date columns are in the correct datetime format for accurate calculations
transactions['day'] = pd.to_datetime(transactions['day'])
repayments['repaymentDay'] = pd.to_datetime(repayments['repaymentDay'])

In [8]:
# Query 1: Calculate current outstanding principal amount for every loan
def calculate_outstanding_principal(transactions, repayments):
    # Calculate the total principal expected from each transaction
    transactions['total_principal'] = transactions['emiPrincipal'] * transactions['tenure'] + transactions['downPaymentAmount']
    
    # Aggregate repayments made by each transactionId
    repayments_summary = repayments.groupby('transactionId')['amount'].sum().reset_index()
    
    # Merge transactions with repayments to calculate outstanding amounts
    merged_df = pd.merge(transactions, repayments_summary, on='transactionId', how='left').fillna(0)
    
    # Calculate the outstanding principal and convert it to rupees (from paise)
    merged_df['outstandingAmount'] = (merged_df['total_principal'] - merged_df['amount']) / 100
    
    return merged_df[['transactionId', 'outstandingAmount']]



In [9]:
# Output for Query 1
outstanding_principal_df = calculate_outstanding_principal(transactions, repayments)
print("Outstanding Principal Amount per Loan:")
print(outstanding_principal_df.head())


Outstanding Principal Amount per Loan:
   transactionId  outstandingAmount
0         704663               0.00
1        1403240               0.00
2        1961345               0.00
3        1682956               0.00
4        2392323            2811.34


In [13]:
# Convert paise to rupees
transactions['emiPrincipal'] = transactions['emiPrincipal'] / 100
transactions['downPaymentAmount'] = transactions['downPaymentAmount'] / 100
repayments['amount'] = repayments['amount'] / 100

# Convert date columns to datetime
transactions['day'] = pd.to_datetime(transactions['day'])
repayments['repaymentDay'] = pd.to_datetime(repayments['repaymentDay'])

# Calculate the total principal expected from each transaction
transactions['total_principal'] = (transactions['emiPrincipal'] * transactions['tenure']) + transactions['downPaymentAmount']

def calculate_aum(transactions, repayments, as_of_date):
    as_of_date = pd.to_datetime(as_of_date)
    
    # Ensure repaymentDay column is timezone-naive
    repayments['repaymentDay'] = repayments['repaymentDay'].dt.tz_localize(None)
    
    # Filter repayments made on or before the specified date
    relevant_repayments = repayments[repayments['repaymentDay'] <= as_of_date]
    
    # Aggregate repayments made up to the specified date
    repayments_summary = relevant_repayments.groupby('transactionId')['amount'].sum().reset_index()
    
    # Merge with transactions to calculate outstanding amounts
    merged_df = pd.merge(transactions, repayments_summary, on='transactionId', how='left')
    
    # Handle missing repayments (no repayments made for some loans)
    merged_df['amount'] = merged_df['amount'].fillna(0)
    
    # Calculate the outstanding principal amount
    merged_df['outstandingAmount'] = merged_df['total_principal'] - merged_df['amount']
    
    # Calculate the AUM (sum of outstanding amounts)
    aum = merged_df['outstandingAmount'].sum()
    
    return round(aum, 2)

# Example: Calculate AUM as of '2023-12-31'
aum_as_of_date = calculate_aum(transactions, repayments, '2023-12-31')
print(f"AUM as of 2023-12-31: {aum_as_of_date} Rupees")

AUM as of 2023-12-31: 304.51 Rupees


In [17]:
import pandas as pd

# Load datasets
transactions = pd.read_csv('https://websdk-assets.s3.ap-south-1.amazonaws.com/public/txns+(13).csv')
repayments = pd.read_csv('https://websdk-assets.s3.ap-south-1.amazonaws.com/public/repayments+(2).csv')

# Convert paise to rupees
transactions['emiPrincipal'] = transactions['emiPrincipal'] / 100
transactions['downPaymentAmount'] = transactions['downPaymentAmount'] / 100
repayments['amount'] = repayments['amount'] / 100

# Convert date columns to datetime and ensure they are timezone-naive
transactions['day'] = pd.to_datetime(transactions['day']).dt.tz_localize(None)
repayments['repaymentDay'] = pd.to_datetime(repayments['repaymentDay']).dt.tz_localize(None)

# Calculate the total principal expected from each transaction
transactions['total_principal'] = (transactions['emiPrincipal'] * transactions['tenure']) + transactions['downPaymentAmount']

# Generate EMI dates
def generate_emi_dates(row):
    return [row['day'] + pd.DateOffset(days=30 * i) for i in range(int(row['tenure']))]

# Apply function to generate EMI dates
emi_dates = transactions.apply(lambda row: generate_emi_dates(row), axis=1)
emi_dates_columns = [f'emi_date_{i+1}' for i in range(max(transactions['tenure']))]
emi_dates_df = pd.DataFrame(emi_dates.tolist(), columns=emi_dates_columns)
transactions = pd.concat([transactions, emi_dates_df], axis=1)

# Melt the DataFrame to have one row per EMI date
emi_dates_columns = [col for col in transactions.columns if col.startswith('emi_date_')]
transactions = transactions.melt(id_vars=transactions.columns.difference(emi_dates_columns), 
                                  value_vars=emi_dates_columns, 
                                  var_name='emi_index', 
                                  value_name='emi_date').drop('emi_index', axis=1)

# Calculate DPD for each EMI
repayment_max_date = repayments['repaymentDay'].max()
transactions['dpd'] = (repayment_max_date - transactions['emi_date']).dt.days

# Categorize into DPD buckets
def categorize_dpd(dpd):
    if pd.isna(dpd):
        return 'Not due'
    elif dpd < 0:
        return 'Not due'
    elif 0 <= dpd < 30:
        return 'DPD 0-30'
    elif 30 <= dpd < 60:
        return 'DPD 30-60'
    elif 60 <= dpd < 90:
        return 'DPD 60-90'
    else:
        return 'DPD 90+'

transactions['dpd_bucket'] = transactions['dpd'].apply(categorize_dpd)

# Merge with repayments to get actual amounts
merged_df = pd.merge(transactions, repayments, on='transactionId', how='left')

# Aggregate amounts by cohort and DPD bucket
cohort_performance = merged_df.groupby([merged_df['day'].dt.to_period('M'), 'dpd_bucket'])['amount'].sum().unstack().fillna(0)

# Rename columns and reset index
cohort_performance = cohort_performance.rename(columns={
    'Not due': 'amount_not_due',
    'DPD 0-30': 'amount_dpd_0_30',
    'DPD 30-60': 'amount_dpd_30_60',
    'DPD 60-90': 'amount_dpd_60_90',
    'DPD 90+': 'amount_dpd_90_above'
}).reset_index()

# Rename the cohort column
cohort_performance.rename(columns={'day': 'month'}, inplace=True)

print("Portfolio Performance by Cohort:")
print(cohort_performance)


Portfolio Performance by Cohort:
dpd_bucket    month  amount_dpd_0_30  amount_dpd_30_60  amount_dpd_60_90  \
0           2022-02             0.00              0.00              0.00   
1           2022-03             0.00              0.00              0.00   
2           2022-04             0.00              0.00              0.00   
3           2022-05             0.00              0.00              0.00   
4           2022-06             0.00              0.00              0.00   
5           2022-07             0.00              0.00              0.00   
6           2022-08             0.00              0.00              0.00   
7           2022-09             0.00              0.00              0.00   
8           2022-10             0.00              0.00              0.00   
9           2022-11             0.00              0.00              0.00   
10          2023-01             0.00              0.00              0.00   
11          2023-02             0.00              0.00 